In [1]:
!pip install efficientnet-pytorch

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16521 sha256=8d073156f2999086b49796dfdb4d571bdca0fca46cf2d73b1e996145dd2eed8d
  Stored in directory: c:\users\youss\appdata\local\pip\cache\wheels\03\3f\e9\911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
Successfully built efficientnet-pytorch


  DEPRECATION: Building 'efficientnet-pytorch' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'efficientnet-pytorch'. Discussion can be found at https://github.com/pypa/pip/issues/6334


In [1]:
# Import necessary libraries
import torch
import torchvision
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
import numpy as np
from efficientnet_pytorch import EfficientNet


In [2]:
# Define transformations for training and validation
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.2),
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1), scale=(0.8, 1.2)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


In [3]:
# Load datasets
train_dataset = datasets.ImageFolder('DATA/train', transform=train_transform)
val_dataset = datasets.ImageFolder('DATA/testing', transform=val_transform)
class_names = train_dataset.classes  # ['Acne', 'Actinic Keratosis', 'Basal Cell Carcinoma', 'Eczemaa', 'Rosacea']

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [4]:
# Load pre-trained EfficientNet-B0 and modify the final layer
model = EfficientNet.from_pretrained('efficientnet-b0')
model._fc = nn.Linear(model._fc.in_features, 5)  # 5 disease classes
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3)


Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b0-355c32eb.pth" to C:\Users\youss/.cache\torch\hub\checkpoints\efficientnet-b0-355c32eb.pth


100.0%


Loaded pretrained weights for efficientnet-b0


In [5]:
# Training function
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=15):
    best_acc = 0.0
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
        # Validation
        model.eval()
        correct = 0
        total = 0
        val_loss = 0.0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        acc = 100 * correct / total
        val_loss = val_loss / len(val_loader)
        scheduler.step(val_loss)
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader):.4f}, '
              f'Val Loss: {val_loss:.4f}, Val Accuracy: {acc:.2f}%')
        if acc > best_acc:
            best_acc = acc
            torch.save(model.state_dict(), 'disease_classifier.pth')


In [6]:
# Evaluation function with probability distribution
def evaluate_model(model, val_loader, class_names, threshold=0.85):
    model.eval()
    correct = 0
    total = 0
    results = []
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            probabilities = torch.softmax(outputs, dim=1)
            for i in range(images.size(0)):
                true_label = labels[i].item()
                probs = probabilities[i].cpu().numpy()
                pred_label = np.argmax(probs)
                is_correct = probs[true_label] >= threshold
                total += 1
                if is_correct:
                    correct += 1
                # Format probability output
                prob_output = ', '.join([f'{class_names[j]}: {probs[j]*100:.2f}%' for j in range(len(class_names))])
                results.append({
                    'true_label': class_names[true_label],
                    'predicted_label': class_names[pred_label],
                    'probabilities': prob_output,
                    'correct': is_correct
                })
    accuracy = 100 * correct / total
    return results, accuracy


In [7]:
# Train the model
print("Training the classifier...")
train_model(model, train_loader, val_loader, criterion, optimizer, scheduler)


Training the classifier...
Epoch 1/15, Loss: 1.2429, Val Loss: 1.6565, Val Accuracy: 44.86%
Epoch 2/15, Loss: 0.7345, Val Loss: 2.0440, Val Accuracy: 54.59%
Epoch 3/15, Loss: 0.4735, Val Loss: 1.8370, Val Accuracy: 59.46%
Epoch 4/15, Loss: 0.3494, Val Loss: 2.5703, Val Accuracy: 61.62%
Epoch 5/15, Loss: 0.2878, Val Loss: 2.8409, Val Accuracy: 59.46%
Epoch 6/15, Loss: 0.1968, Val Loss: 2.5547, Val Accuracy: 63.78%
Epoch 7/15, Loss: 0.1616, Val Loss: 2.1275, Val Accuracy: 65.95%
Epoch 8/15, Loss: 0.1212, Val Loss: 1.7456, Val Accuracy: 69.73%
Epoch 9/15, Loss: 0.1364, Val Loss: 1.4883, Val Accuracy: 70.81%
Epoch 10/15, Loss: 0.1229, Val Loss: 1.2490, Val Accuracy: 71.89%
Epoch 11/15, Loss: 0.0746, Val Loss: 1.0878, Val Accuracy: 72.97%
Epoch 12/15, Loss: 0.0989, Val Loss: 0.9772, Val Accuracy: 74.05%
Epoch 13/15, Loss: 0.1095, Val Loss: 0.9345, Val Accuracy: 75.68%
Epoch 14/15, Loss: 0.0574, Val Loss: 0.9008, Val Accuracy: 77.30%
Epoch 15/15, Loss: 0.0865, Val Loss: 0.8250, Val Accuracy:

In [9]:
# Load the best model
model.load_state_dict(torch.load('disease_classifier.pth'))
model.to(device)

# Evaluate on test set
print("\nEvaluating on test set...")
results, accuracy = evaluate_model(model, val_loader, class_names)
print(f"\nTest Accuracy (threshold >= 85%): {accuracy:.2f}%")
print("\nSample Predictions:")
for i, result in enumerate(results[:5]):  # Show first 5 predictions
    print(f"Image {i+1}:")
    print(f"True Label: {result['true_label']}")
    print(f"Predicted Label: {result['predicted_label']}")
    print(f"Probabilities: {result['probabilities']}")
    print(f"Correct: {result['correct']}")
    print()


Evaluating on test set...

Test Accuracy (threshold >= 85%): 69.19%

Sample Predictions:
Image 1:
True Label: Acne
Predicted Label: Eczemaa
Probabilities: Acne: 2.48%, Actinic Keratosis: 0.06%, Basal Cell Carcinoma: 0.02%, Eczemaa: 97.40%, Rosacea: 0.03%
Correct: False

Image 2:
True Label: Acne
Predicted Label: Eczemaa
Probabilities: Acne: 6.88%, Actinic Keratosis: 0.30%, Basal Cell Carcinoma: 0.09%, Eczemaa: 92.70%, Rosacea: 0.03%
Correct: False

Image 3:
True Label: Acne
Predicted Label: Acne
Probabilities: Acne: 99.91%, Actinic Keratosis: 0.02%, Basal Cell Carcinoma: 0.02%, Eczemaa: 0.01%, Rosacea: 0.03%
Correct: True

Image 4:
True Label: Acne
Predicted Label: Eczemaa
Probabilities: Acne: 4.79%, Actinic Keratosis: 0.33%, Basal Cell Carcinoma: 0.34%, Eczemaa: 94.13%, Rosacea: 0.41%
Correct: False

Image 5:
True Label: Acne
Predicted Label: Acne
Probabilities: Acne: 99.99%, Actinic Keratosis: 0.00%, Basal Cell Carcinoma: 0.00%, Eczemaa: 0.00%, Rosacea: 0.00%
Correct: True

